In [31]:
"""
Перевірка гіпотези про відповідність  розподілу неперервного параметра деякому закону
"""
import numpy as np
import scipy
from pandas import read_csv
from scipy.special import comb
from scipy.stats import norm, lognorm

data = read_csv('./src/lab2_data2.csv', sep=';')

alpha = 0.95

doctor_collumn_name = 'DR1'
class_count = 4
doctors = [data[data[doctor_collumn_name] == i][doctor_collumn_name].count() for i in range(1, 1 + class_count)]
total_count = sum(doctors)

p = 1.0 / class_count
expected = p * total_count

#Для даних із табл. Б.2 перевірити гіпотезу про те, що кількість пацієнтів, що обстежувалися в кожного з лікарів, підпорядкована рівномірному закону розподілу.
x2 = sum((((s - expected) ** 2) / expected for s in doctors))
x2_table = scipy.stats.chi2.ppf(alpha, class_count - 1)

H0 = 'Кількість пацієнтів, котра обстежувалась у кожного лікаря підпорядкована рівномірному розподілу'
H1 = 'Розподіл пацієнтів, які обстежувались у кожного лікаря не є рівномірним'

print(H0 if x2_table < x2 else H1)
print('\n')


#Перевірити гіпотези про те, що величина SYST розподілена нормально і логнормально (табл. Б.4)

class Search:
    data_split_count = 20
    alph = 0.95

    def __init__(self, data) -> None:
        super().__init__()
        mean = np.mean(data)
        # нормализируем данные
        self.data = [x / mean for x in data]
        self.total_data_length = len(data)
        self.data_split_count = self._get_spkit_count(data)

    @staticmethod
    def _get_spkit_count(data) -> int:
        return int((len(data) * 10) ** (1 / 3))

    def is_normal(self) -> bool:
        return self._check_for_distribution(norm.pdf)

    def is_lognormal(self) -> bool:
        return self._check_for_distribution(lambda x: lognorm.pdf(x=x, s=1))

    def _check_for_distribution(self, distribution) -> bool:
        histogram, borders = np.histogram(self.data, self.data_split_count)
        dist = distribution(x=borders)
        espect = [self.total_data_length * (dist[i] - dist[i + 1]) for i in range(len(dist) - 1)]
        hi_sqr = sum([((espect[i] - histogram[i]) ** 2) / espect[i] for i in range(len(espect))])
        tabl_hi = scipy.stats.chi2.ppf(self.alph, self.data_split_count - 1)
        return hi_sqr > tabl_hi


search = Search(list(data['Syst1']))
H0 = 'Данна послідновність(тиск крові) розподілена нормально'
H1 = 'Невірно,  що ця послідовність(тиск крові) розподілена нормально'
print(H0 if search.is_normal() else H1)

H0 = 'Данна послідновність(тиск крові) розподілена логонормально'
H1 = 'Невірно,  що ця послідовність(тиск крові) розподілена логонормально'
print(H0 if search.is_lognormal() else H1)


Кількість пацієнтів, котра обстежувалась у кожного лікаря підпорядкована рівномірному розподілу


Данна послідновність(тиск крові) розподілена нормально
Данна послідновність(тиск крові) розподілена логонормально
